In [ ]:
!pip3 install --upgrade --user --quiet google-cloud-aiplatform

## Restart the runtime after package install.

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

In [ ]:
import sys

if 'google.colab' in sys.modules:
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)


**The kernel needs to restart. Please wait until it is finished before continuing to the next step.**

## Authenticate the colab environment.

In [ ]:
import sys

if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Set Google Cloud project information and initialize Vertex AI SDK



In [ ]:
PROJECT_ID = "trv-hs-src-data-lake-prod" # @param {type: "string"}
REGION = "us-central1" # @param {type: "string"}

import vertexai

# Initialize the Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

## Import libraries

In [ ]:
import IPython.display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from vertexai.genereative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Image
)

import json
from os import path
from glob import glob

from PIL import Image as PillowImage


In [ ]:
MODEL_ID = "gemini-1.5-pro-preview-0409" # @param {type: 'string'}
BASE_IMAGE_PATH = "/app/_assets/" # @param {type: 'string'}

In [ ]:
PROMPT = """
    Hi,
    please act as a hotel quality assistant.
    Analyse the image or images of this hotel objectively.
    You must only respond with a json object.

    Your json object must cover the following fields:
    [{
        "file_name": string,
        "location": 'indoor' | 'outdoor',
        "room_type": string,
        "ambience": string,
        "image_quality": number, // score from 0 to 1 in double decimal
        "image_rating": number, // score from 0 to 1 in double decimal
    }, ...]

    Sort your output from the most appealing pictures to the least appealing,
    factoring in the overall_rating.

"""

In [ ]:
"""
Lists all the hotel directories in the _assets folder - each directory has the name of a hotel
Returns a list of directories containing images with the key as the file-name and the value as Image class
"""

class HotelAnalyer:
    def __init__(self, images_path: str) -> None:
        vertexai.init(project=PROJECT_ID, location=REGION)
        self.images_path = images_path + '/*'
        self.model = GenerativeModel(MODEL_NAME)

    def list_hotel_directories(self):
        print('Listing hotel directories...')

        directories = glob(self.images_path)

        hotel_images = {}

        for  (hotel_directory_index, hotel_directory) in enumerate(directories):
            image_paths = glob(hotel_directory + '/*')
            image_names = [image_path.split('/')[-1] for image_path in image_paths]
            hotel_names = [hotel_dir.split('/')[-1] for hotel_dir in directories]
            hotel_name = hotel_names[hotel_directory_index]

            ### only for testing
            for image in image_paths:
                image_instace = PillowImage.open(image)
                print(image, image_instace.size)
            ###

            print(f'Loading images for {hotel_name}...')
            # get list of loaded image objects using google aiplatform sdk
            images = [Image.load_from_file(image) for image in image_paths]

            # create the list of hotel images, per hotel as object key
            for image_index in range(0, len(images)):
                if not hotel_images.get(hotel_name):
                    hotel_images[hotel_name] = []

                # crete the image list alternating between file names and actual image data
                hotel_images[hotel_name].append(image_names[image_index])
                hotel_images[hotel_name].append(images[image_index])

        return hotel_images

    def analyse_hotel_images(self, hotel_name: str, hotel_images: list):
        print(f'Analyzing images for {hotel_name}...')

        response = self.model.generate_content(
            [PROMPT, 'hotel_name: ' + hotel_name, *hotel_images], generation_config={
                'response_mime_type': 'application/json'})

        print(response)

    def start(self):
        print('Starting...')

        hotels = self.list_hotel_directories()

        for hotel_name, hotel_images in hotels.items():
            self.analyse_hotel_images(hotel_name, hotel_images)


def main():
    analyser = HotelAnalyer(BASE_IMAGE_PATH)
    analyser.start()


if __name__ == '__main__':
    main()